In [ ]:
import pyarrow.parquet as pq

import pandas as pd
from pathlib import Path
import seaborn as sns

# Basic exploration of pageview data
## With one of the pageview-files

In [ ]:
df = pd.read_parquet('Drive Daten/students_pageviews_2021-01-01.parquet.gzip', engine='pyarrow')

In [ ]:
df[df['time_engaged_in_s'] > 300].sort_values(by="time_engaged_in_s")

In [ ]:
df['time_engaged_in_s'].max()

In [ ]:
df['time_engaged_in_s'].quantile(.90)

In [ ]:
sns.histplot(df.loc[df['time_engaged_in_s']<df['time_engaged_in_s'].quantile(.90), 'time_engaged_in_s'],bins=100)

In [ ]:
df['time_engaged_in_s']

In [ ]:
len(df)

In [ ]:
bins = [-1,4,29,59, 300_000]
labels = ["<5", "<30", "<60", ">60"]
df["time_spent_cut"] = pd.cut(df['time_engaged_in_s'], bins, labels=labels)
df.time_spent_cut.unique()

In [ ]:
df

### Make a pivot table from the data since a groupby doesn't work that well with non-numerical data

In [ ]:
pivot_df_time_cuts = df[["article_drive_id", "time_engaged_in_s", "time_spent_cut"]].pivot_table(index="article_drive_id" ,columns="time_spent_cut", aggfunc="count")

pivot_df_time_cuts = pivot_df_time_cuts.reset_index()

pivot_df_time_cuts.columns = pivot_df_time_cuts.columns.droplevel()
pivot_df_time_cuts.columns = ["article_drive_id"] + list(pivot_df_time_cuts.columns[1:])
pivot_df_time_cuts

In [ ]:
df_read = df[df['time_engaged_in_s']>0]
df_read

In [ ]:
len(pd.unique(df.article_drive_id))

In [ ]:
pd.unique(df.article_drive_id)

In [ ]:
df["count"] = 1

In [ ]:
df[["article_drive_id", "count"]].groupby('article_drive_id').sum().reset_index()

## Now: Exploration of article data

In [ ]:
df = pd.read_parquet('Techlabs/students_articles_enriched_1.parquet.gzip', engine='pyarrow')
df.columns

# Read ALL the DATA

## First: article data

In [ ]:
drivepath = Path("Techlabs")
df_article_list = []

relevant_columns = [ "publisher_id" , "article_drive_id" , "published_at_local" ,"locality" , "topic" , "genre" ]


for count, curpath in enumerate(drivepath.glob("students_articles_enriched_*")):
    df = pd.read_parquet(curpath, engine='pyarrow')
    general_table = df[relevant_columns]
    df_article_list.append(general_table)

df_allarticles = pd.concat(df_article_list)
del df_article_list

In [ ]:
# df_allarticles.to_parquet("all_articles.parquet.gzip", compression="gzip")

In [ ]:
# df_allarticles.to_csv("all_articles.csv")

## Then: Pageview Data

In [ ]:
df_user_list = []
drivepath = Path("Drive Daten")

len_count = len(list(drivepath.rglob("students_pageviews*")))


import_columns = ["article_drive_id", "time_engaged_in_s"]


for count, curpath in enumerate(drivepath.rglob("students_pageviews*")):
#     print(f"{count+1}/{len_count}")
    df = pd.read_parquet(curpath, columns=import_columns, engine='pyarrow')

    df_user_list.append(df)

In [ ]:
df_user_intermediate = pd.concat(df_user_list)
del df_user_list
df_user_intermediate.to_parquet("pageviews_intermediate_time_engaged.parquet")

In [ ]:
bins = [-1,4,29,59, 300_000]
labels = ["<5", "<30", "<60", ">60"]

relevant_columns = ["article_drive_id", "time_engaged_in_s", "time_spent_cut"]

df_user_intermediate["time_spent_cut"] = pd.cut(df_user_intermediate['time_engaged_in_s'], bins, labels=labels)


In [ ]:
pivot_df_time_cuts = df_user_intermediate[relevant_columns].pivot_table(index="article_drive_id" ,columns="time_spent_cut", aggfunc="count")

In [ ]:
pivot_df_time_cuts

In [ ]:
# create new index
pivot_df_time_cuts = pivot_df_time_cuts.reset_index()
# format new index of pivot table like old index with "article_drive_id" as the first column name
pivot_df_time_cuts.columns = pivot_df_time_cuts.columns.droplevel()
pivot_df_time_cuts.columns = ["article_drive_id"] + list(pivot_df_time_cuts.columns[1:])
pivot_df_time_cuts["count_views"] = pivot_df_time_cuts["<5"] + pivot_df_time_cuts["<30"] + pivot_df_time_cuts["<60"] + pivot_df_time_cuts[">60"]
pivot_df_time_cuts

### Write out the pivot_df as a file

In [ ]:
# pivot_df_time_cuts.to_parquet("pageviews_groupby_watchtime.parquet.gzip", compression="gzip")

In [ ]:
# pivot_df_time_cuts.to_csv("pageviews_groupby_watchtime.csv")

### Build complete Dataframe from user_pivot_table and df_allarticles

In [ ]:
df_complete = df_allarticles.merge(pivot_df_time_cuts, on="article_drive_id")
df_complete.head()

In [ ]:
df_complete.to_pickle("aggregate_data")

#### Code to check validity

In [ ]:
df_allusers.loc[df_allusers.article_drive_id == "7ca7a1ca182332beef95b03281317e92"]

In [ ]:
df_allarticles[df_allarticles.article_drive_id == "7ca7a1ca182332beef95b03281317e92"]

In [ ]:
df_all_users_articles = df_allarticles.merge(df_allusers, on="article_drive_id")
df_all_users_articles.head()

In [ ]:
df_all_users_articles[df_all_users_articles["article_drive_id"] == "397d5f5ee004e751c1c8ab46ed4af498"]

## Exploring "weird" article data with exceptionally high number of pageviews

In [ ]:
drivepath = Path("Techlabs")
df_article_fulltext_list = []

relevant_columns = [ "article_drive_id" ,"locality" , "topic" , "genre", "article_header", "article_full_text" ]


for count, curpath in enumerate(drivepath.glob("students_articles_enriched_*")):
    df = pd.read_parquet(curpath, engine='pyarrow')
    general_table = df[relevant_columns]
    df_article_fulltext_list.append(general_table)

df_allarticles_fulltext = pd.concat(df_article_fulltext_list)

In [ ]:
df_allarticles_fulltext.loc[df_allarticles_fulltext["article_drive_id"] == "7ca7a1ca182332beef95b03281317e92", "article_full_text"]

In [ ]:
df_allarticles_fulltext.loc[df_allarticles_fulltext["article_drive_id"] == "397d5f5ee004e751c1c8ab46ed4af498", "article_full_text"]

## Crosscheck_user_viewcounts

In [ ]:
df_users_crosscheck_list = []
drivepath = Path("Drive Daten")

columns_to_check = ["article_drive_id"]

for count, curpath in enumerate(drivepath.rglob("students_pageviews*")):
    df = pd.read_parquet(curpath, columns=columns_to_check, engine='pyarrow')
    df["count"] = 1
    df_users_crosscheck_list.append(df)

In [ ]:
crosscheck_df = pd.concat(df_article_crosscheck_list)
# Corona-Artikel
len(crosscheck_df[crosscheck_df["article_drive_id"] == "7ca7a1ca182332beef95b03281317e92"])

In [ ]:
# "Vierte Laden-Musik"
len(crosscheck_df[crosscheck_df["article_drive_id"] == "397d5f5ee004e751c1c8ab46ed4af498"])